In [2]:
import pandas as pd
import s3fs
import os


# Configuration MinIO
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},skip_instance_cache=True)
parquet_files = fs.ls("aymericp/SDV_CNAM_parquet")
# Bucket Parquet
input_bucket = 'aymericp/SDV_CNAM_parquet'
output_csv_path = './output/concat_sdv_filtered.csv'

# Création du dossier de sortie
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

# Liste des fichiers parquet
parquet_files = fs.ls(input_bucket)
dfs = []

for file_path in parquet_files:
    if file_path.endswith('.parquet'):
        print(f"🔄 Lecture du fichier : {file_path}")
        try:
            with fs.open(file_path, 'rb') as f:
                df = pd.read_parquet(f, columns=['asu_nat', 'rem_mon', 'prs_nat', 'l_prs_nat','rem_date','act_dnb'])

            # ✅ Filtrage combiné :
            df_filtered = df[
                (df['asu_nat'] == 4) &
                (df['prs_nat'].astype(str).str.startswith('6'))
            ][['rem_mon', 'prs_nat', 'l_prs_nat','rem_date','act_dnb']]

            dfs.append(df_filtered)

        except Exception as e:
            print(f"⚠️ Erreur sur {file_path} : {e}")

# Concaténation et export
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv(output_csv_path, index=False)
    print(f"✅ Fichier CSV sauvegardé localement à : {output_csv_path}")
else:
    print("⚠️ Aucun fichier valide ou aucune ligne correspondante trouvée.")


🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2010.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2011.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2012.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2013.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2014.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2015.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2016.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2017.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2018.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2019.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2020.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2021.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2022.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2023.parquet
🔄 Lecture du fichier : aymericp/SDV_CNAM_parquet/NTOT2024.parquet
✅ Fichier 

In [4]:
import pandas as pd

# Charger le CSV
df = pd.read_csv('./output/concat_sdv_filtered.csv')

# GroupBy l_prs_nat et rem_date et sommer rem_mon
df_grouped = df.groupby(['l_prs_nat', 'rem_date'], as_index=False)[['rem_mon','act_dnb']].sum()

# Sauvegarder le résultat
df_grouped.to_csv('./output/rem_mon_grouped.csv', index=False)
print("✅ Fichier groupé sauvegardé : ./output/rem_mon_grouped.csv")

# Extraire l'année depuis rem_date (supposé au format AAAAMMJJ ou similaire)
df['annee'] = df['rem_date'].astype(str).str[:4]

# GroupBy l_prs_nat et annee, puis sommer rem_mon
df_grouped_a = df.groupby(['l_prs_nat', 'annee'], as_index=False)[['rem_mon','act_dnb']].sum()

# Sauvegarder le résultat
df_grouped_a.to_csv('./output/rem_mon_grouped_by_annee.csv', index=False)
print("✅ Fichier groupé par année sauvegardé : ./output/rem_mon_grouped_by_annee.csv")


✅ Fichier groupé sauvegardé : ./output/rem_mon_grouped.csv
✅ Fichier groupé par année sauvegardé : ./output/rem_mon_grouped_by_annee.csv


In [7]:
df_grouped_a['tarif'] = df_grouped_a['rem_mon']/df_grouped_a['act_dnb']
df_grouped_a.to_csv('./output/rem_mon_grouped_by_annee_tarifs.csv', index=False)

In [6]:
df_grouped

,l_prs_nat,rem_date,rem_mon,act_dnb,tarif
0,IJ CONGE PATHOLOGIQUE PAMC,201707,5.514000e+01,1.000000e+00,55.140000
1,IJ MAJOREES -3MOIS,200701,0.000000e+00,0.000000e+00,NaN
2,IJ MAJOREES -3MOIS,201001,1.606213e+08,3.293484e+06,48.769426
3,IJ MAJOREES -3MOIS,201002,1.445520e+08,2.954483e+06,48.926321
4,IJ MAJOREES -3MOIS,201003,1.706301e+08,3.481119e+06,49.015892
...,...,...,...,...,...
1261,INDEMNITE TEMPORAIRE D'INAPTITUDE AT/MP,202408,1.854611e+06,3.341700e+04,55.499019
1262,INDEMNITE TEMPORAIRE D'INAPTITUDE AT/MP,202409,1.910585e+06,3.389900e+04,56.361100
1263,INDEMNITE TEMPORAIRE D'INAPTITUDE AT/MP,202410,1.987581e+06,3.534100e+04,56.240099
1264,INDEMNITE TEMPORAIRE D'INAPTITUDE AT/MP,202411,1.888234e+06,3.330700e+04,56.691794
